# Chapter 3: Coding attention mechanisms

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights

In [ ]:
import torch

In [ ]:
# embedding vectors

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

attn_scores_2

In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
attn_weights_2_tmp, attn_weights_2_tmp.sum()

In [ ]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [ ]:
attn_weights_2_naive = softmax_naive(attn_scores_2)
attn_weights_2_naive, attn_weights_2_naive.sum()

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
attn_weights_2, attn_weights_2.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(attn_weights_2_tmp.shape[0]), attn_weights_2_tmp,label='sum')
plt.plot(range(attn_weights_2.shape[0]), attn_weights_2,label='softmax')
plt.legend()
plt.show()

In [ ]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

context_vec_2

### 3.3.2 Computing attention weights for all input tokens

In [ ]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

attn_scores

In [ ]:
attn_scores = inputs @ inputs.T
attn_scores

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

In [ ]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

## 3.4 Implementing self-attention with trainable weights

### 3.4.1 Computing the attention weights step by step

In [ ]:
x_2 = inputs[1] # single element from input embedding sequence
d_in = inputs.shape[1] # input embedding size
d_out = 2 # output embedding size

In [ ]:
torch.manual_seed(123)

In [ ]:
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
x_2.shape, W_query.shape

`x_2 @ W_query` is the same as $x_2^{T}W_{query}$

In [ ]:
# PyTorch treats a 1D vector of shape (3,) as a row vector when used on the left of @.
query_2 = x_2 @ W_query # (1x3) x (3x2) => 1x2 => torch.Size([2])
query_2.shape

In [ ]:
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

In [ ]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
attn_score_22

In [ ]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

In [ ]:
d_k = keys.shape[-1]
d_k

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
attn_weights_2

In [ ]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

### 3.4.2 Implementing a compact self-attention Python class

In [ ]:
import torch.nn as nn

In [ ]:
# NOTE: this is of course scaled dot product self-attention

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [ ]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
sa_v1(inputs)

In [ ]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T # omega

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2(inputs)

**Exercise 3.1 Comparing SelfAttention_v1 and SelfAttention_v2**

Note that `nn.Linear` in `SelfAttention_v2` uses a different weight initialization scheme as `nn.Parameter(torch.rand(d_in, d_out))` used in `SelfAttention_v1`, which causes both mechanisms to produce different results. To check that both implementations, `SelfAttention_v1` and `SelfAttention_v2`, are otherwise similar, we can transfer the weight matrices from a `SelfAttention_v2` object to a `SelfAttention_v1`, such that both objects then produce the same results.

Your task is to correctly assign the weights from an instance of `SelfAttention_v2` to an instance of `SelfAttention_v1`. To do this, you need to understand the relationship between the weights in both versions. (Hint: nn.Linear stores the weight matrix in a transposed form.) After the assignment, you should observe that both instances produce the same outputs.

In [ ]:
# compare W_query from v1 and v2 SelfAttention
# sa_v1.W_query

In [ ]:
# sa_v2.W_query.weight

In [ ]:
# sa_v1.W_query.T

In [ ]:
# sa_v2.W_query.weight = nn.Parameter(sa_v1.W_query.T)

# sa_v2.W_query.weight

In [ ]:
# sa_v2.W_key.weight = nn.Parameter(sa_v1.W_key.T)
# sa_v2.W_value.weight = nn.Parameter(sa_v1.W_value.T)

In [ ]:
# sa_v2(inputs)

## 3.5 Hiding future words with causal attention

### 3.5.1 Applying a causal attention mask

In [ ]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

attn_weights 

In [ ]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple

In [ ]:
masked_simple = attn_weights*mask_simple
masked_simple

In [ ]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
masked

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
attn_weights

### 3.5.2 Masking additional attention weights with dropout

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
dropout(example)

In [ ]:
torch.manual_seed(123)
dropout(attn_weights)

### 3.5.3 Implementing a compact causal attention class

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)

        # avoid device mismatch
        # will move to CPU or GPU when the model is moved
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # because we now have a batch dimension

        # .masked_fill_ trailing underscore are operations done in place
        # no copies are made
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)

        # scaled attention
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
context_vecs.shape

## 3.6 Extending single-head attention to multi-head attention

### 3.6.1 Stacking multiple single-head attention layers


In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
    dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in, d_out, context_length, dropout, qkv_bias
            )
            for _ in range(num_heads)]
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

**Exercise 3.2 Returning two-dimensional embedding vectors**

Change the input arguments for the `MultiHeadAttentionWrapper(..., num_heads=2)` call such that the output context vectors are two-dimensional instead of four dimensional while keeping the setting `num_heads=2`. Hint: You don’t have to modify the class implementation; you just have to change one of the other input arguments.

In [ ]:
ex_mha = MultiHeadAttentionWrapper(
    d_in=3, d_out=1, 
    context_length=context_length,dropout=0.0,
    num_heads=2
)
ex_context_vecs = ex_mha(batch)
ex_context_vecs.shape

### 3.6.2 Implementing multi-head attention with weight splits

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # compute output size of each head

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)

        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )

        self.out_proj = nn.Linear(d_out, d_out) # linear layer combines head outputs

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys       = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values   = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys       = keys.transpose(1, 2) # (b, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1, 2)
        values   = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) # (b, num_tokens, n_heads, head_dim)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out) # combine heads

        context_vec = self.out_proj(context_vec)
        return context_vec

In [ ]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],
                    
                    [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

In [ ]:
a.shape

In [ ]:
a.transpose(2, 3).shape

In [ ]:
result = a @ a.transpose(2, 3)
result

In [ ]:
result.shape

In [ ]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)
second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

In [ ]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
context_vecs

In [ ]:
context_vecs.shape

**Exercise 3.3 Initializing GPT-2 size attention modules**

Using the `MultiHeadAttention` class, initialize a multi-head attention module that has the same number of attention heads as the smallest GPT-2 model (12 attention heads). Also ensure that you use the respective input and output embedding sizes similar to GPT-2 (768 dimensions). Note that the smallest GPT-2 model supports a context length of 1,024 tokens.

In [ ]:
768 / 12

In [ ]:
gpt2_mha = MultiHeadAttention(d_in=768,d_out=768,context_length=1024,dropout=0.1,num_heads=12)

In [ ]:
gpt2_inputs = torch.rand((64, 1024, 768))

In [ ]:
gpt2_context_vecs = gpt2_mha(gpt2_inputs)

In [ ]:
gpt2_context_vecs.shape